In [20]:
import pandas as pd
import numpy as np
from collections import Counter
from sklearn.preprocessing import LabelEncoder
import pandas_profiling as ppf

####  ================================== step_1/5 读取数据时将ID作为index ========================================

In [3]:
# base信息
base_train_sum=pd.read_csv(r'./data/base_train_sum.csv',encoding='gb2312').reset_index(drop=True).set_index('ID')
base_varify=pd.read_csv(r'./data/base_verify1.csv',encoding='gb2312').reset_index(drop=True).set_index('ID')
knowledge_train=pd.read_csv(r'./data/knowledge_train_sum.csv',encoding='gb2312').reset_index(drop=True).set_index('ID')
paient_information_verify1=pd.read_csv(r'./data/paient_information_verify1.csv',encoding='gb2312').reset_index(drop=True).set_index('ID')
# money信息
money_report_train_sum=pd.read_csv(r'./data/money_report_train_sum.csv',encoding='gb2312').reset_index(drop=True).set_index('ID')
money_information_verify1=pd.read_csv(r'./data/money_information_verify1.csv',encoding='gb2312').reset_index(drop=True).set_index('ID')
year_report_train_sum=pd.read_csv(r'./data/year_report_train_sum.csv',encoding='gb2312').reset_index(drop=True).set_index('ID')
year_report_verify1=pd.read_csv(r'./data/year_report_verify1.csv',encoding='gb2312').reset_index(drop=True).set_index('ID')


####  ========================================= step_2/5 数据合并 ==============================================

In [4]:
# 基本信息的数据
no_year=pd.concat([pd.concat([base_train_sum,base_varify],sort=True).drop(columns='控制人ID'), pd.concat([knowledge_train,paient_information_verify1],sort=True)],axis=1,sort=True)
# no_year

In [5]:
# 三年的有关money的数据
three_year=pd.concat([pd.concat([money_report_train_sum,money_information_verify1]),pd.concat([year_report_train_sum,year_report_verify1]).drop(columns='year')],axis=1)

####  ================================== step_3/5 对money的three_year处理 ========================================

In [6]:
# 把‘year’补充完整,此步较慢，可以保存文件至three_year
three_year['year']=three_year['year'].fillna(0)
pp=[]
for i in list(set(three_year.index)):
    a=three_year[three_year.index==i]['year'].iloc[0]
    b=three_year[three_year.index==i]['year'].iloc[1]
    c=three_year[three_year.index==i]['year'].iloc[2]
    year=[2015,2016,2017]
    real=[a,b,c]
    if 0 not in real:
        continue
    else:
        pp.append(i)

df=three_year[three_year.index==pp[0]]
for i in range(1,len(pp)):
    df=pd.concat([three_year[three_year.index==pp[i]],df])
    
df['year']=[2015.,2016.,2017.]*int(4035/3)
three_year.combine_first(df)
# three_year.to_csv('./data/three_year.csv')
three_year.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 137802 entries, 28 to 5970000
Data columns (total 18 columns):
year           137802 non-null float64
债权融资额度         136464 non-null float64
债权融资成本         136418 non-null float64
股权融资额度         136450 non-null float64
股权融资成本         136414 non-null float64
内部融资和贸易融资额度    136484 non-null float64
内部融资和贸易融资成本    136410 non-null float64
项目融资和政策融资额度    136406 non-null float64
项目融资和政策融资成本    136392 non-null float64
从业人数           136477 non-null float64
资产总额           136345 non-null float64
负债总额           136482 non-null float64
营业总收入          136382 non-null float64
主营业务收入         136444 non-null float64
利润总额           136471 non-null float64
净利润            136369 non-null float64
纳税总额           136487 non-null float64
所有者权益合计        136418 non-null float64
dtypes: float64(18)
memory usage: 25.0 MB


In [7]:
# 三年数据分别处理，留下三年ID的交集
money_2015=three_year[three_year['year']==2015].drop(['year'],axis=1)
money_2016=three_year[three_year['year']==2016].drop(['year'],axis=1)
money_2017=three_year[three_year['year']==2017].drop(['year'],axis=1)
inner0=list(set(money_2015.index).intersection(set(money_2016.index)).intersection(set(money_2017.index)))  # money的交集
inner=list(set(no_year.index).intersection(set(inner0)))                                                    # base的交集
print(len(inner0),len(inner))

44589 44587


In [8]:
# 所有空值暂且用平均数填充（所有者权益总额=资产总额-负债总额）
lt=[]  # 记录每一年的数据的形状
for money in [money_2015,money_2016,money_2017]:
    mean=money.describe()
    values={money.columns[0]:mean[mean.columns[0]][1], money.columns[1]:mean[mean.columns[1]][1], money.columns[2]:mean[mean.columns[2]][1], money.columns[3]:mean[mean.columns[3]][1],
           money.columns[4]:mean[mean.columns[4]][1],money.columns[5]:mean[mean.columns[5]][1],money.columns[6]:mean[mean.columns[6]][1],money.columns[7]:mean[mean.columns[7]][1],
           money.columns[8]:mean[mean.columns[8]][1],money.columns[9]:mean[mean.columns[9]][1],money.columns[10]:mean[mean.columns[10]][1],money.columns[11]:mean[mean.columns[11]][1],
           money.columns[12]:mean[mean.columns[12]][1],money.columns[13]:mean[mean.columns[13]][1],money.columns[14]:mean[mean.columns[14]][1],money.columns[15]:mean[mean.columns[15]][1],
           money.columns[16]:mean[mean.columns[16]][1]}
    money.fillna(value=values,inplace=True)
    money['所有者权益合计']=money['资产总额']-money['负债总额']
#     舍弃交集外的ID
    for index in money.index:
        if index in inner:
            continue
        else:
            money.drop(index,axis=0,inplace=True) # inplace指定原地操作
    lt.append(money.shape)
#     对每一年的数据每一列标准化(x-mu)/std
    for i in range(money.shape[1]):
        money[money.columns[i]]=(money[money.columns[i]] - mean[mean.columns[i]][1])/mean[mean.columns[i]][2]
print(lt)

[(44587, 17), (44587, 17), (44587, 17)]


In [9]:
# 三年的数据暂且取平均
average=(money_2015+money_2016+money_2017)/3
average

,债权融资额度,债权融资成本,股权融资额度,股权融资成本,内部融资和贸易融资额度,内部融资和贸易融资成本,项目融资和政策融资额度,项目融资和政策融资成本,从业人数,资产总额,负债总额,营业总收入,主营业务收入,利润总额,净利润,纳税总额,所有者权益合计
ID,,,,,,,,,,,,,,,,,
28,-0.376171,-0.376457,0.405896,0.406650,-0.226785,-0.226242,-0.340208,-0.340444,0.064210,-0.584066,-0.399117,-0.338235,-0.384190,-0.247130,-0.392001,-0.463028,-0.005115
230,-0.376171,-0.376457,-0.232576,-0.232517,-0.132010,-0.131411,-0.284146,-0.284366,-0.101589,-0.454216,-0.371305,-0.547330,-0.504989,-0.402584,-0.305447,-0.463028,0.106610
429,0.422541,0.422178,1.588908,1.590641,-0.325227,-0.325101,-0.340208,-0.340444,-0.214133,1.632982,1.380157,1.567048,1.398623,0.636495,-1.183199,-0.463028,-0.461403
727,-0.376171,-0.376457,2.127535,2.127238,0.274559,0.276155,-0.340208,-0.340444,0.356377,1.912166,2.116246,1.688369,1.048483,0.836141,-1.225835,-0.463028,-1.435530
1137,-0.024247,-0.024672,-0.285557,-0.285544,-0.213995,-0.213397,-0.235906,-0.236193,-0.322280,-0.215560,-0.362734,-0.451093,-0.505011,-0.277347,-0.345359,-0.463028,0.393309
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5999995,2.476890,2.476638,0.248212,0.248808,-0.325227,-0.325101,0.004820,0.004681,-0.031999,1.666780,0.436294,0.657269,0.712197,1.231301,0.221042,1.236277,1.261471
5999996,-0.254412,-0.254746,-0.176810,-0.176847,-0.325227,-0.325101,-0.340208,-0.340444,0.685207,-0.866247,-0.730358,-0.695970,-0.661647,-0.581006,-0.121579,-0.356253,0.239803
5999997,0.126371,0.126091,-0.285557,-0.285544,0.710330,0.713966,-0.340208,-0.340444,-0.863629,-0.224643,-0.293843,0.119754,0.067854,0.304117,0.567990,0.853867,0.253745


####  ========================================= step_4/5 对base的no_year处理 =======================================

In [10]:
for index in no_year.index:
        if index in inner:
            continue
        else:
            no_year.drop(index,axis=0,inplace=True) # inplace指定原地操作

In [11]:
no_year.info()

<class 'pandas.core.frame.DataFrame'>
Float64Index: 44587 entries, 28.0 to 5999999.0
Data columns (total 11 columns):
flag       34584 non-null float64
企业类型       44141 non-null object
区域         44179 non-null object
控制人持股比例    44143 non-null float64
控制人类型      44100 non-null object
注册时间       44115 non-null float64
注册资本       44136 non-null float64
行业         44183 non-null object
专利         44144 non-null float64
商标         44122 non-null float64
著作权        44165 non-null float64
dtypes: float64(7), object(4)
memory usage: 4.1+ MB


In [12]:
# 以众数填充
lt2=[]
No_year=no_year.drop(columns='flag')
for i in No_year.columns:
    lt2.append(list(Counter(no_year[i]).most_common(1)[0])[0])

values={'企业类型':lt2[0],'区域':lt2[1],'控制人持股比例':lt2[2],'控制人类型':lt2[3],'注册时间':lt2[4],
        '注册资本':lt2[5],'行业':lt2[6],'专利':lt2[7],'商标':lt2[8],'著作权':lt2[9]}
No_year.fillna(value=values,inplace=True)

In [13]:
no_year=no_year.combine_first(No_year)

In [14]:
# 注册时间以2000年为元年，注册资本标准化(x-mu)/std
no_year['注册时间']=no_year['注册时间']-2000
no_year['注册资本']=(no_year['注册资本']-no_year['注册资本'].mean())/no_year['注册资本'].std()

In [15]:
# 对文本数据编码
usa=no_year[['行业','企业类型','控制人类型','区域']]
for col in ['行业','企业类型','控制人类型','区域']:
    no_year[col] = LabelEncoder().fit_transform(no_year[col])
    print(Counter(usa[col]))
    print(Counter(no_year[col]))

Counter({'交通运输业': 7871, '商业服务业': 7425, '服务业': 7382, '零售业': 7314, '工业': 7300, '社区服务': 7295})
Counter({0: 7871, 1: 7425, 3: 7382, 5: 7314, 2: 7300, 4: 7295})
Counter({'合伙企业': 9318, '有限责任公司': 8849, '农民专业合作社': 8822, '集体所有制企业': 8806, '股份有限公司': 8792})
Counter({1: 9318, 2: 8849, 0: 8822, 4: 8806, 3: 8792})
Counter({'企业法人': 22578, '自然人': 22009})
Counter({0: 22578, 1: 22009})
Counter({'江西': 6843, '广西': 6412, '福建': 6305, '山东': 6273, '湖北': 6268, '广东': 6260, '湖南': 6226})
Counter({3: 6843, 2: 6412, 6: 6305, 0: 6273, 4: 6268, 1: 6260, 5: 6226})


###### 行业
商业服务业：1 ；服务业：3 ；社区服务：4 ；零售业：5 ；工业：2 ；交通运输业:0
###### 区域
福建：   6  ；广西：2  ； 江西：  3  ； 山东：    0  ；湖北： 4  ；湖南：5 ；广东：1 
###### 企业类型
农民专业合作社：0；股份有限公司：3；有限责任公司：2；集体所有制企业：4；合伙企业：1
###### 控制人类型
自然人:1     ;企业法人:0

In [16]:
no_year

,flag,专利,企业类型,区域,商标,控制人持股比例,控制人类型,注册时间,注册资本,著作权,行业
ID,,,,,,,,,,,
28.0,1.0,0.0,0,6,1.0,0.56,0,7.0,-1.036650,1.0,0
230.0,1.0,0.0,0,1,0.0,1.00,0,8.0,-0.577361,0.0,3
429.0,1.0,1.0,4,3,0.0,0.75,1,5.0,1.634936,0.0,2
727.0,1.0,0.0,3,3,0.0,0.54,0,1.0,1.301864,0.0,5
1137.0,1.0,0.0,2,1,0.0,0.83,0,2.0,-0.970035,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...
5999995.0,0.0,0.0,1,5,1.0,0.83,0,0.0,1.627924,1.0,2
5999996.0,0.0,1.0,0,4,1.0,0.93,1,11.0,-0.994577,0.0,5
5999997.0,0.0,1.0,4,6,1.0,0.73,0,13.0,1.410551,0.0,0


#### =================================== step_5/5 合并base与money ==============================================

In [17]:
encoded_all_data=pd.concat([no_year,average],axis=1,sort=True)
encoded_all_data

,flag,专利,企业类型,区域,商标,控制人持股比例,控制人类型,注册时间,注册资本,著作权,...,项目融资和政策融资成本,从业人数,资产总额,负债总额,营业总收入,主营业务收入,利润总额,净利润,纳税总额,所有者权益合计
ID,,,,,,,,,,,,,,,,,,,,,
28.0,1.0,0.0,0,6,1.0,0.56,0,7.0,-1.036650,1.0,...,-0.340444,0.064210,-0.584066,-0.399117,-0.338235,-0.384190,-0.247130,-0.392001,-0.463028,-0.005115
230.0,1.0,0.0,0,1,0.0,1.00,0,8.0,-0.577361,0.0,...,-0.284366,-0.101589,-0.454216,-0.371305,-0.547330,-0.504989,-0.402584,-0.305447,-0.463028,0.106610
429.0,1.0,1.0,4,3,0.0,0.75,1,5.0,1.634936,0.0,...,-0.340444,-0.214133,1.632982,1.380157,1.567048,1.398623,0.636495,-1.183199,-0.463028,-0.461403
727.0,1.0,0.0,3,3,0.0,0.54,0,1.0,1.301864,0.0,...,-0.340444,0.356377,1.912166,2.116246,1.688369,1.048483,0.836141,-1.225835,-0.463028,-1.435530
1137.0,1.0,0.0,2,1,0.0,0.83,0,2.0,-0.970035,0.0,...,-0.236193,-0.322280,-0.215560,-0.362734,-0.451093,-0.505011,-0.277347,-0.345359,-0.463028,0.393309
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5999995.0,0.0,0.0,1,5,1.0,0.83,0,0.0,1.627924,1.0,...,0.004681,-0.031999,1.666780,0.436294,0.657269,0.712197,1.231301,0.221042,1.236277,1.261471
5999996.0,0.0,1.0,0,4,1.0,0.93,1,11.0,-0.994577,0.0,...,-0.340444,0.685207,-0.866247,-0.730358,-0.695970,-0.661647,-0.581006,-0.121579,-0.356253,0.239803
5999997.0,0.0,1.0,4,6,1.0,0.73,0,13.0,1.410551,0.0,...,-0.340444,-0.863629,-0.224643,-0.293843,0.119754,0.067854,0.304117,0.567990,0.853867,0.253745


In [19]:
encoded_all_data.to_csv(r'./data/created_data/encoded_all_data.csv')

In [21]:
ppf.ProfileReport(encoded_all_data)